In [41]:
# Disable Cache
import os, sys
sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUGINFO"] = "1"
os.environ["NUMBA_DEBUG"] = "1"
os.environ["NUMBA_DEVELOPER_MODE"] = "1"
os.environ["NUMBA_SHOW_HELP"] = "1"

import quantfreedom as qf
import pandas as pd
from numba import literal_unroll, njit
from quantfreedom.backtester.nb.simulate import simulate_from_signals
from quantfreedom.backtester.base.base import rsi_below_entries
from quantfreedom.backtester.enums.enums import (
    LeverageMode, 
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
    )
# qf.clear_cache()

prices = pd.read_csv('30min.csv', index_col='time')
entries = rsi_below_entries(
    timeperiods=[20],
    below_ranges=[50],
    prices=prices.close
)

In [42]:
df, order_rec, cart = simulate_from_signals(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=3.,
    risk_rewards=[4],
    size_pct=1.,
    sl_pcts=[2],
    # gains_pct_filter=100,
    sl_to_be=True,
    sl_to_be_based_on=SL_BE_or_Trail_BasedOn.high_price,
    sl_to_be_zero_or_entry=0,
    sl_to_be_when_pct_from_avg_entry=3,
)

In [43]:
pd.DataFrame(df)

,or_set,ind_set,total_trades,total_BE,gains_pct,win_rate,to_the_upside,total_fees,total_pnl,ending_eq
0,0,0,145.0,0,68.080581,20.0,0.025275,NaN,680.8065,1680.805812


In [44]:
pd.DataFrame(order_rec).head(50)

,avg_entry,bar,equity,fees_paid,ind_set,or_set,order_id,order_type,price,real_pnl,size_value,sl_prices,tp_prices,tsl_prices
0,10850.000000,31,1000.000000,NaN,0,0,0,0.0,10850.000000,NaN,471.965263,10633.000000,11783.139083,NaN
1,10787.137891,32,1000.000000,NaN,0,0,1,0.0,10725.000000,NaN,471.965263,10571.395133,11714.870607,NaN
2,10787.137891,32,980.000000,1.121389,0,0,2,4.0,10571.395133,-20.0000,943.930527,10571.395133,11714.870607,NaN
3,10577.500000,33,980.000000,NaN,0,0,3,0.0,10577.500000,NaN,462.525958,10365.950000,11487.203102,NaN
4,10550.179617,34,980.000000,NaN,0,0,4,0.0,10523.000000,NaN,462.525958,10339.176024,11457.533067,NaN
5,10538.761751,35,980.000000,NaN,0,0,5,0.0,10516.000000,NaN,462.525958,10327.986516,11445.133223,NaN
6,10538.761751,436,1097.600000,1.736695,0,0,6,5.0,11445.133223,117.6000,1387.577874,10327.986516,11445.133223,NaN
7,11356.000000,478,1097.600000,NaN,0,0,7,0.0,11356.000000,NaN,518.029073,11128.880000,12332.656906,NaN
8,11334.208263,479,1097.600000,NaN,0,0,8,0.0,11312.500000,NaN,518.029073,11107.524097,12308.991001,NaN
9,11323.953653,480,1097.600000,NaN,0,0,9,0.0,11303.500000,NaN,518.029073,11097.474580,12297.854458,NaN


In [45]:
pd.DataFrame(cart).T

,0
leverage,NaN
max_equity_risk_pct,3.0
max_equity_risk_value,NaN
risk_rewards,4.0
size_pct,1.0
size_value,NaN
sl_pcts,2.0
sl_prices,NaN
sl_to_be,True
sl_to_be_based_on,1.0
